# Eligibility for mobilization: Cohort ID and Discretizing script

Author: Kaveri Chhikara
v1: October 30, 2024
v2: February 10, 2025

This script identifies the cohort using CLIF 2.0 tables and discretizes the dataset at an hourly level

 
                        🚨Code will break if the following requirements are not satisfied🚨  
#### Requirements:
* Required table filenames should be `clif_patient`, `clif_hospitalization`, `clif_adt`, `clif_vitals`, `clif_labs`, `clif_medication_admin_continuous`, `clif_respiratory_support`
* Within each table, the following variables and categories are required.

| Table Name | Required Variables | Required Categories |
| --- | --- | --- |
| **patient** | `patient_id`, `race_category`, `ethnicity_category`, `sex_category`, `death_dttm` | - |
| **hospitalization** | `patient_id`, `hospitalization_id`, `admission_dttm`, `discharge_dttm`, `age_at_admission` | - |
| **adt** |  `hospitalization_id`, `hospital_id`,`in_dttm`, `out_dttm`, `location_category` | - |
| **vitals** | `hospitalization_id`, `recorded_dttm`, `vital_category`, `vital_value` | heart_rate, resp_rate, sbp, dbp, map, spo2, weight_kg, height_cm |
| **labs** | `hospitalization_id`, `lab_result_dttm`, `lab_category`, `lab_value` | lactate |
| **medication_admin_continuous** | `hospitalization_id`, `admin_dttm`, `med_name`, `med_category`, `med_dose`, `med_dose_unit` | norepinephrine, epinephrine, phenylephrine, vasopressin, dopamine, angiotensin(optional), nicardipine, nitroprusside, clevidipine, cisatracurium, vecuronium, rocuronium |
| **respiratory_support** | `hospitalization_id`, `recorded_dttm`, `device_category`, `mode_category`, `tracheostomy`, `fio2_set`, `lpm_set`, `resp_rate_set`, `peep_set`, `resp_rate_obs` | - |


Updates 2/10:
* Get discharge_dttm and death_dttm. Everyone in the cohort must have one of these. If not dead, assume discharged alive.
* Include all paralytics in the mCIDE. Instead of excluding anyone who ever received a paralytics, create flags for paralytics. While creating flags for eligibility, exclude hours when the patient was on a paralytic.
* Update exclusion criteria - exclude all patients intubated for < 4 hrs instead of 2 hrs to be consistent with the cool-off period.
* Add code to stitch encounters when there are multiple hospitals at a site
* Extend the analysis to competing risk. Events - 1 - eligible, 2- died, 3 discharged-alive

## Load Libraries

In [1]:
#! pip install pandas numpy duckdb seaborn matplotlib plotly
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import pyCLIF

## import outlier json
with open('../config/outlier_config.json', 'r') as f:
    outlier_cfg = json.load(f)

Loaded configuration from config.json
{'site_name': 'UCMC', 'tables_path': '/Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19', 'file_type': 'parquet'}


## Required columns and categories

In [2]:
rst_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'device_name',
    'device_category',
    'mode_name', 
    'mode_category',
    'tracheostomy',
    'fio2_set',
    'lpm_set',
    'resp_rate_set',
    'peep_set',
    'resp_rate_obs'
]

vitals_required_columns = [
    'hospitalization_id',
    'recorded_dttm',
    'vital_category',
    'vital_value'
]
vitals_of_interest = ['heart_rate', 'respiratory_rate', 'sbp', 'dbp', 'map', 'spo2', 'weight_kg', 'height_cm']

labs_required_columns = [
    'hospitalization_id',
    'lab_result_dttm',
    'lab_category',
    'lab_value',
    'lab_value_numeric'
]
labs_of_interest = ['lactate']

meds_required_columns = [
    'hospitalization_id',
    'admin_dttm',
    'med_name',
    'med_category',
    'med_dose',
    'med_dose_unit'
]
meds_of_interest = [
    'norepinephrine', 'epinephrine', 'phenylephrine', 'vasopressin',
    'dopamine', 'angiotensin', 'nicardipine', 'nitroprusside',
    'clevidipine', 'cisatracurium', 'vecuronium', 'rocuronium '
]

## Load data

In [3]:
patient = pyCLIF.load_data('clif_patient')
hospitalization = pyCLIF.load_data('clif_hospitalization')
adt = pyCLIF.load_data('clif_adt')

# ensure id variable is of dtype character
hospitalization['hospitalization_id']= hospitalization['hospitalization_id'].astype(str)
patient['patient_id']= patient['patient_id'].astype(str)
adt['hospitalization_id']= adt['hospitalization_id'].astype(str)

Data loaded successfully from /Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19/clif_patient.parquet
Data loaded successfully from /Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19/clif_hospitalization.parquet
Data loaded successfully from /Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19/clif_adt.parquet


## Duplicate check

If duplicates exist, only the first row is preserved after arranging the data by time. Please check your CLIF tables if there are duplicates. 

In [4]:
# check for duplicates
# patient table should be unique by patient id
patient = pyCLIF.remove_duplicates(patient, ['patient_id'], 'patient')
# hospitalization table should be unique by hospitalization id
hospitalization = pyCLIF.remove_duplicates(hospitalization, ['hospitalization_id'], 'hospitalization')
# adt table should be unique by hospitalization id and in dttm
adt = pyCLIF.remove_duplicates(adt, ['hospitalization_id', 'hospital_id', 'in_dttm'], 'adt')

Processing DataFrame: patient
No duplicates found based on columns: ['patient_id'].
Processing DataFrame: hospitalization
No duplicates found based on columns: ['hospitalization_id'].
Processing DataFrame: adt
No duplicates found based on columns: ['hospitalization_id', 'hospital_id', 'in_dttm'].


In [5]:
print(f"Total Number of unique encounters in the hospitalization table: {pyCLIF.count_unique_encounters(hospitalization, 'hospitalization_id')}")

Total Number of unique encounters in the hospitalization table: 448402


In [6]:
# Standardize all _dttm variables to the same format
patient = pyCLIF.standardize_datetime_utc(patient, 'death_dttm')
hospitalization = pyCLIF.standardize_datetime_utc(hospitalization, ['admission_dttm', 'discharge_dttm'])
adt = pyCLIF.standardize_datetime_utc(adt, ['in_dttm', 'out_dttm'])

## Cohort Identification

**Inclusion Criteria:**

* Adult admissions between 2020-03-01 and 2022-03-31
* Encounters receiving invasive mechanical ventilation during this period

**Exclusion criteria:**

1. Encounters that were on vent for less than 4 hours in the first 72 hours of first intubation
2. Encounters that were on trach in the first 72 hours of first intubation

In [7]:
# setting up a dictionary to keep track of STROBE counts
strobe_counts = {}

#### (A) Date and Age Filter

In [8]:
# STEP A: Basic Data Cleaning + Date/Age Filter
#   - Filter hospitalization for date range & adult patients
#   - Then reduce ADT to those hospitalization_ids


print("\n=== STEP A: Filter by date range & age ===\n")
date_mask = (hospitalization['admission_dttm'] >= '2020-03-01') & \
            (hospitalization['admission_dttm'] <= '2022-03-31')
age_mask = (hospitalization['age_at_admission'] >= 18)

hospitalization_cohort = hospitalization[date_mask & age_mask].copy()

strobe_counts['A_after_date_age_filter'] = hospitalization_cohort['hospitalization_id'].nunique()
print(f"Number of unique hospitalizations after date & age filter: {strobe_counts['A_after_date_age_filter']}")


=== STEP A: Filter by date range & age ===

Number of unique hospitalizations after date & age filter: 164613


#### (B) Stitch hospitalizations

Combine multiple `hospitalization_ids` into a single `encounter_block` for patients who transfer between hospital campuses or return soon after discharge. Hospitalizations that have a gap of **6 hours or less** between the discharge dttm and admission dttm are put in one encounter block.

In [9]:
# Filter ADT to only those in the cohort set
cohort_ids = hospitalization_cohort['hospitalization_id'].unique().tolist()
adt_cohort = adt[adt['hospitalization_id'].isin(cohort_ids)].copy()

In [10]:
# STEP B: Stitch Encounters => 'encounter_block'
# Use stitch_encounters from pyCLIF with time_interval=6

print("\n=== STEP B: Stitch encounters ===\n")
stitched_cohort = pyCLIF.stitch_encounters(hospitalization_cohort, adt_cohort, time_interval=6)
# stitched_cohort now has: 'patient_id','hospitalization_id','encounter_block' etc. This will have duplicate rows because of location category
# We only want 1 row per unique encounter_block for the next steps.
stitched_unique = stitched_cohort[['patient_id', 'encounter_block']].drop_duplicates()

strobe_counts['B_before_stitching'] = stitched_cohort['hospitalization_id'].nunique()
strobe_counts['B_after_stitching'] = stitched_unique['encounter_block'].nunique()
strobe_counts['B_stitched_hosp_ids'] = strobe_counts['B_before_stitching']-strobe_counts['B_after_stitching']
print(f"Number of unique hospitalizations before stitching: {stitched_cohort['hospitalization_id'].nunique()}")
print(f"Number of unique encounter blocks after stitching: {strobe_counts['B_after_stitching']}")
print(f"Number of linked hospitalization ids: {strobe_counts['B_before_stitching']-strobe_counts['B_after_stitching']}")


=== STEP B: Stitch encounters ===



/Users/kavenchhikara/Desktop/CLIF/CLIF-eligibility-for-mobilization/code/pyCLIF.py:503: FutureWarning: The 'downcast' keyword in bfill is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  hospital_block['encounter_block'] = hospital_block['encounter_block'].bfill(downcast='int')


Number of unique hospitalizations before stitching: 164613
Number of unique encounter blocks after stitching: 164315
Number of linked hospitalization ids: 298


#### (C) Outcome dttm for each encounter block

Calulcate final outcome dttm for each encounter block using discharge or death times.   
We drop hospitalizations where `discharge_dttm` and `death_dttm` are missing.

In [11]:
# STEP C: Merge with patient to define discharge/death
#    each encounter_block’s final outcome time

print("\n=== STEP C: Identify final outcome times (discharge vs death) ===\n")

# 1) Merge hospital & patient on hospitalization_id
#    (Though we’ll eventually group on encounter_block)
tmp_hosp_patient = (
    hospitalization_cohort[['hospitalization_id','patient_id','discharge_dttm']]
    .merge(patient[['patient_id','death_dttm']], on='patient_id', how='left')
)

# 2) Drop any row where both discharge_dttm and death_dttm are missing
mask_no_discharge_no_death = tmp_hosp_patient['discharge_dttm'].isna() & tmp_hosp_patient['death_dttm'].isna()
hosp_no_outcome = tmp_hosp_patient[mask_no_discharge_no_death].copy()
tmp_hosp_patient = tmp_hosp_patient[~mask_no_discharge_no_death].copy()

# 3) Define final_outcome_dttm and a died_flag
tmp_hosp_patient['final_outcome_dttm'] = np.where(
    tmp_hosp_patient['death_dttm'].notna(),
    tmp_hosp_patient['death_dttm'],
    tmp_hosp_patient['discharge_dttm']
)
tmp_hosp_patient['died_flag'] = np.where(tmp_hosp_patient['death_dttm'].notna(), 1, 0)

stitched_cohort_filtered = stitched_cohort[['patient_id', 'hospitalization_id', 'encounter_block',
                                            'location_category', 'in_dttm', 'out_dttm', 
                                            'admission_dttm','discharge_dttm', 
                                            'hospital_id',]].drop_duplicates()

# 4) Merge that into stitched_cohort, so each row knows final_outcome_dttm
stitched_merge = stitched_cohort_filtered.merge(
    tmp_hosp_patient[['hospitalization_id','final_outcome_dttm','died_flag']],
    on='hospitalization_id', how='inner'
).drop_duplicates()

# 5) Group by encounter_block to find final_outcome at encounter block-level:
agg_dict = {
    'final_outcome_dttm': 'max',  # the latest if multiple hospitalizations
    'died_flag': 'max'            # if any hospitalization was a death
}
encounter_block_outcome = stitched_merge.groupby('encounter_block').agg(agg_dict).reset_index()
encounter_block_outcome.rename(columns={'final_outcome_dttm':'encounter_block_outcome_dttm'}, inplace=True)

# 6) Now we have final outcome time at block level
strobe_counts['C_final_encounter_blocks_with_outcome'] = encounter_block_outcome['encounter_block'].nunique()
strobe_counts['C_hosp_without_outcome'] = hosp_no_outcome['hospitalization_id'].nunique()
print(f"Encounter blocks after ensuring each has discharge or death: {strobe_counts['C_final_encounter_blocks_with_outcome']}")
if len(hosp_no_outcome) > 0:
    hosp_no_outcome.to_csv('../output/intermediate/hospitalizations_without_outcomes.csv', index=False)
    print(f"Hospitalizations without death or discharge dttm: {len(hosp_no_outcome)}. \n Check output/intermediate/hospitalizations_without_outcomes.csv")


=== STEP C: Identify final outcome times (discharge vs death) ===

Encounter blocks after ensuring each has discharge or death: 164286
Hospitalizations without death or discharge dttm: 29. 
 Check output/intermediate/hospitalizations_without_outcomes.csv


In [12]:
# Mapping of patient id, hospitalization id and encounter blocks
all_ids = stitched_cohort[
    stitched_cohort['encounter_block'].isin(encounter_block_outcome['encounter_block'])
][['patient_id', 'hospitalization_id', 'encounter_block']].drop_duplicates()
all_ids.shape

(164584, 3)

In [13]:
strobe_counts

{'A_after_date_age_filter': 164613,
 'B_before_stitching': 164613,
 'B_after_stitching': 164315,
 'B_stitched_hosp_ids': 298,
 'C_final_encounter_blocks_with_outcome': 164286,
 'C_hosp_without_outcome': 29}

#### (D) Identify ventilator usage

Filter down to encounters that received invasive mechanical ventilation

In [14]:
# STEP D: Identify Ventilator Usage
# Load respiratory support only for the relevant “hospitalization_id” set
# These hospitalizations map to an encounter_block for final grouping.

print("\n=== STEP D: Load & process respiratory support => Identify IMV usage ===\n")

# 1) Load respiratory support
resp_support_raw = pyCLIF.load_data(
    'clif_respiratory_support',
    columns=rst_required_columns,
    filters={'hospitalization_id': all_ids['hospitalization_id'].unique().tolist()}
)

resp_support = resp_support_raw.copy()
resp_support['recorded_dttm'] = pd.to_datetime(resp_support['recorded_dttm'])
resp_support = pyCLIF.standardize_datetime_utc(resp_support, 'recorded_dttm') #standardize to utc tz naive
resp_support['device_category'] = resp_support['device_category'].str.lower()
resp_support['mode_category'] = resp_support['mode_category'].str.lower()
resp_support['fio2_set'] = pd.to_numeric(resp_support['fio2_set'], errors='coerce')
resp_support['lpm_set'] = pd.to_numeric(resp_support['lpm_set'], errors='coerce')
resp_support['resp_rate_set'] = pd.to_numeric(resp_support['resp_rate_set'], errors='coerce')
resp_support['peep_set'] = pd.to_numeric(resp_support['peep_set'], errors='coerce')
resp_support['resp_rate_obs'] = pd.to_numeric(resp_support['resp_rate_obs'], errors='coerce')

# del resp_support_raw


=== STEP D: Load & process respiratory support => Identify IMV usage ===

Data loaded successfully from /Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19/clif_respiratory_support.parquet


Respiratory Support Summary

In [15]:
results_list = []
group_cols = 'device_category'  # or a list like ['device_category','mode_category']
numeric_cols = ['fio2_set','peep_set','lpm_set', 'resp_rate_set', 'resp_rate_obs']

for col in numeric_cols:
    tmp = pyCLIF.create_summary_table(
        df=resp_support,
        numeric_col=col,
        group_by_cols=group_cols
    )
    # tmp might have columns:
    #   ['device_category','N','missing','min','q25','median','q75','mean','max']
    # Insert a "variable" column next to the group-by columns:
    tmp['variable'] = col
    # We want "device_category" (the group col), then "variable", then the rest
    if isinstance(group_cols, str):
        group_cols_list = [group_cols]  # unify into list
    else:
        group_cols_list = group_cols  # already a list
    # Reorder so that group-by columns come first, then 'variable', then the rest
    front_cols = group_cols_list + ['variable']
    # Build the list of remaining columns
    rest_cols = [c for c in tmp.columns if c not in front_cols]
    new_cols = front_cols + rest_cols
    tmp = tmp[new_cols]
    results_list.append(tmp)

# Finally, concatenate all results
final_summary_resp_support = pd.concat(results_list, ignore_index=True)
final_summary_resp_support.to_csv('../output/final/summary_respiratory_support_by_device.csv', index=False)

In [16]:
results_list = []
group_cols = ['device_category','mode_category']
numeric_cols = ['fio2_set','peep_set','lpm_set', 'resp_rate_set', 'resp_rate_obs']

for col in numeric_cols:
    tmp = pyCLIF.create_summary_table(
        df=resp_support,
        numeric_col=col,
        group_by_cols=group_cols
    )
    # tmp might have columns:
    #   ['device_category','N','missing','min','q25','median','q75','mean','max']
    # Insert a "variable" column next to the group-by columns:
    tmp['variable'] = col
    # We want "device_category" (the group col), then "variable", then the rest
    if isinstance(group_cols, str):
        group_cols_list = [group_cols]  # unify into list
    else:
        group_cols_list = group_cols  # already a list
    # Reorder so that group-by columns come first, then 'variable', then the rest
    front_cols = group_cols_list + ['variable']
    # Build the list of remaining columns
    rest_cols = [c for c in tmp.columns if c not in front_cols]
    new_cols = front_cols + rest_cols
    tmp = tmp[new_cols]
    results_list.append(tmp)

# Finally, concatenate all results
final_summary_resp_support = pd.concat(results_list, ignore_index=True)
final_summary_resp_support.to_csv('../output/final/summary_respiratory_support_by_device_mode.csv', index=False)

In [17]:
# 2) Process with waterfall logic
processed_resp_support = pyCLIF.process_resp_support(resp_support)

Initiating waterfall processing...
Creating recorded_date and recorded_hour...
Sorting data by 'hospitalization_id' and 'recorded_dttm'...
Fixing missing 'device_category' and 'device_name' based on 'mode_category'...
Fixing 'device_category' and 'device_name' based on neighboring records...
Handling duplicates and removing rows with all key variables missing...
Filling forward 'device_category' within each hospitalization...
Creating 'device_cat_id' to track changes in 'device_category'...
Filling 'device_name' within each 'device_cat_id'...
Creating 'device_id' to track changes in 'device_name'...
Filling 'mode_category' within each 'device_id'...
Creating 'mode_cat_id' to track changes in 'mode_category'...
Filling 'mode_name' within each 'mode_cat_id'...
Creating 'mode_name_id' to track changes in 'mode_name'...
Adjusting 'fio2_set' for 'room air' device_category...
Adjusting 'mode_category' for 't-piece' devices...
Filling remaining variables within each 'mode_name_id'...
Filling 

In [18]:
# 3) Merge to get encounter_block for the cohort identified so far
resp_stitched = processed_resp_support.merge(
    all_ids[['hospitalization_id','encounter_block']],
    on='hospitalization_id', how='right'
)

In [19]:
print("\n=== Apply outlier thresholds ===\n")

# (Optional) If FiO2 is >1 on average => scale by /100
fio2_mean = resp_stitched['fio2_set'].mean(skipna=True)
# If the mean is greater than 1, divide 'fio2_set' by 100
if fio2_mean and fio2_mean > 1.0:
    # Only divide values greater than 1 to avoid re-dividing already correct values
    resp_stitched.loc[resp_stitched['fio2_set'] > 1, 'fio2_set'] = \
        resp_stitched.loc[resp_stitched['fio2_set'] > 1, 'fio2_set'] / 100
    print("Updated fio2_set to be between 0.21 and 1")
else:
    print("FIO2_SET mean=", fio2_mean, "is within the required range")

pyCLIF.apply_outlier_thresholds(resp_stitched, 'fio2_set', *outlier_cfg['fio2_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'peep_set', *outlier_cfg['peep_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'lpm_set',  *outlier_cfg['lpm_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'resp_rate_set', *outlier_cfg['resp_rate_set'])
pyCLIF.apply_outlier_thresholds(resp_stitched, 'resp_rate_obs', *outlier_cfg['resp_rate_obs'])


=== Apply outlier thresholds ===

FIO2_SET mean= 0.4798112288188503 is within the required range


In [20]:
# 4) Identify IMV
imv_mask = resp_stitched['device_category'].str.contains("imv", case=False, na=False)
resp_stitched_imv = resp_stitched[imv_mask].copy()
# this creates a on vent field for everytime the patient is on a vent
resp_stitched_imv['on_vent'] = 1


strobe_counts['D_imv_hospitalizations'] = resp_stitched_imv['hospitalization_id'].nunique()
strobe_counts['D_imv_encounter_blocks'] = resp_stitched_imv['encounter_block'].nunique()

print(f"Total IMV respiratory support hospitalizations: {strobe_counts['D_imv_hospitalizations']}")
print(f"Total IMV respiratory support encounter blocks: {strobe_counts['D_imv_encounter_blocks']}")

Total IMV respiratory support hospitalizations: 5690
Total IMV respiratory support encounter blocks: 5690


#### (E) Vent start and end times 

Calculate vent start times for the first episode of invasive mechanical intubation.   
Limitation: the vent end time might not be associated with the same intubation episode.

In [22]:
# STEP E: Determine Vent Start/End for Each Hospitalization and Encounter block

print("\n=== STEP E: Determine ventilation times (start/end) at hospitalization level and encounter block level ===\n")

# at the hospitalization id level
vent_start_end = resp_stitched_imv.groupby('hospitalization_id').agg(
    vent_start_time=('recorded_dttm','min'),
    vent_end_time=('recorded_dttm','max')
).reset_index()

# Exclude edge case: if start_time == end_time 
# these would otherwise have been excluded when we remove encounters on vent for less than 4 hours
check_same_vent_start_end = vent_start_end[vent_start_end['vent_start_time'] == vent_start_end['vent_end_time']].copy()
vent_start_end= vent_start_end[vent_start_end['vent_start_time'] != vent_start_end['vent_end_time']].copy()

strobe_counts['E_hospitalizations_with_valid_vent'] = vent_start_end['hospitalization_id'].nunique()
strobe_counts['E_hospitalizations_with_same_vent_start_end'] = check_same_vent_start_end['hospitalization_id'].nunique()
print(f"Unique hospitalizations with valid IMV start/end: {strobe_counts['E_hospitalizations_with_valid_vent']}")

# at the block level
block_vent_times = resp_stitched_imv.groupby('encounter_block', dropna=True).agg(
    vent_start_time=('recorded_dttm','min'),
    vent_end_time=('recorded_dttm','max')
).reset_index()

# If start==end, no real vent- there was just ONE vent entry, this exclusion can count under 
block_same_vent = block_vent_times[block_vent_times['vent_start_time']==block_vent_times['vent_end_time']].copy()
block_vent_times = block_vent_times[block_vent_times['vent_start_time']!=block_vent_times['vent_end_time']].copy()

strobe_counts['E_blocks_with_valid_vent'] = block_vent_times['encounter_block'].nunique()
strobe_counts['E_blocks_with_same_vent_start_end'] = block_same_vent['encounter_block'].nunique()
print(f"Unique encounter blocks with valid IMV start/end: {strobe_counts['E_blocks_with_valid_vent']}")

valid_blocks_vent = block_vent_times['encounter_block'].unique()

# Filter all_ids to only keep rows where encounter_block is in valid_blocks_vent
all_ids = all_ids[all_ids['encounter_block'].isin(valid_blocks_vent)]


=== STEP E: Determine ventilation times (start/end) at hospitalization level and encounter block level ===

Unique hospitalizations with valid IMV start/end: 5452
Unique encounter blocks with valid IMV start/end: 5452


In [23]:
strobe_counts

{'A_after_date_age_filter': 164613,
 'B_before_stitching': 164613,
 'B_after_stitching': 164315,
 'B_stitched_hosp_ids': 298,
 'C_final_encounter_blocks_with_outcome': 164286,
 'C_hosp_without_outcome': 29,
 'D_imv_hospitalizations': 5690,
 'D_imv_encounter_blocks': 5690,
 'F_hospitalizations_with_valid_vent': 5452,
 'F_hospitalizations_with_same_vent_start_end': 238,
 'E_blocks_with_valid_vent': 5452,
 'E_blocks_with_same_vent_start_end': 238,
 'E_hospitalizations_with_valid_vent': 5452,
 'E_hospitalizations_with_same_vent_start_end': 238}

#### (F) Hourly Sequence 

This section achieves the following important steps:  
* Identifies the first and last recorded times for vitals for each encounter block
* These times are used to generate an hourly sequence of patients hospitalization journey
* Combines with hourly vent usage data from the respiratory support table
* Excludes encounters on vent for less than 4 hours in the first 72 hours
* Excludes encounters on trach in the first 72 hours 
* Creates a final dataframe with the identified cohort

In [24]:
# STEP F: Generate Hourly Sequence & Exclude encounter blocks with <4 Vent Hours
#  Create an hourly timeline from vent_start to last vital or outcome time for each encounter block
# We stop operating at hospitalization id level 

print("\n=== STEP G: Hourly sequence generation & < 4 hour vent exclusion BLOCK level===\n")

# 1) define the 'end_time' for the sequence from vitals or outcome.
vitals_cohort = pyCLIF.load_data('clif_vitals',
    columns=vitals_required_columns,
    filters={'hospitalization_id': all_ids['hospitalization_id'].unique().tolist(), 
             'vital_category': vitals_of_interest}
)
vitals_cohort['recorded_dttm'] = pd.to_datetime(vitals_cohort['recorded_dttm'])
vitals_cohort = pyCLIF.standardize_datetime_utc(vitals_cohort, 'recorded_dttm')


=== STEP G: Hourly sequence generation & < 4 hour vent exclusion BLOCK level===

Data loaded successfully from /Users/kavenchhikara/Desktop/CLIF/CLIF-UCMC/rclif/c19/clif_vitals.parquet


In [25]:
summary_vitals = pyCLIF.create_summary_table(
        df=vitals_cohort,
        numeric_col='vital_value',
        group_by_cols='vital_category'
    )
summary_vitals.to_csv('../output/final/summary_vitals_by_category.csv', index=False)

In [26]:
# Merge to get encounter_block on each vital
vitals_stitched = vitals_cohort.merge(all_ids, on='hospitalization_id', how='left')
# Group by block => find earliest & latest vital for that block
vital_bounds_block = vitals_stitched.groupby('encounter_block', dropna=True)['recorded_dttm'].agg(['min','max']).reset_index()
vital_bounds_block.columns = ['encounter_block','block_first_vital_dttm','block_last_vital_dttm']

# 2) Merge block_vent_times with vital_bounds_block
final_blocks = block_vent_times.merge(vital_bounds_block, on='encounter_block', how='inner')

In [ ]:
# 3) If block_last_vital_dttm < vent_start_time => weird edge case. Ideally shouldn't happen. 
# If such bad blocks exist, check your CLIF tables bro
bad_block = final_blocks[final_blocks['block_last_vital_dttm'] < final_blocks['vent_start_time']]
if len(bad_block) > 0:
    print("Warning: Some blocks have last vital < vent start:\n", bad_block)
else:
    print("There are no bad blocks! Good job CLIF-ing")

In [27]:
# 4) Generate the hourly sequence at block level
def generate_hourly_sequence_block(row):
    blk  = row['encounter_block'].iloc[0]
    start_time = row['vent_start_time'].iloc[0]
    end_time   = row['block_last_vital_dttm'].iloc[0]
    hourly_timestamps = pd.date_range(start=start_time, end=end_time, freq='h')
    return pd.DataFrame({
        'encounter_block': blk,
        'recorded_dttm': hourly_timestamps
    })

hourly_seq_block = final_blocks.groupby('encounter_block', as_index=False).apply(generate_hourly_sequence_block)
hourly_seq_block = hourly_seq_block.reset_index(drop=True)

# hourly_seq_block['recorded_dttm'] = hourly_seq_block['recorded_dttm'].dt.tz_convert('UTC')
hourly_seq_block['recorded_date'] = hourly_seq_block['recorded_dttm'].dt.date
hourly_seq_block['recorded_hour'] = hourly_seq_block['recorded_dttm'].dt.hour

/var/folders/zj/jxkvz04s3q55f82vwjbj_5w80000gq/T/ipykernel_92784/2900972920.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  hourly_seq_block = final_blocks.groupby('encounter_block', as_index=False).apply(generate_hourly_sequence_block)


In [28]:
# Check for duplicates in hourly_seq_block- could be because of DST, so we have converted all tz to UTC
# if duplicates exist, check why
hourly_seq_block_check = pyCLIF.remove_duplicates(hourly_seq_block, ['encounter_block', 'recorded_date', 'recorded_hour'], 
                                                  'hour_sequence_check')
del hourly_seq_block_check

Processing DataFrame: hour_sequence_check
No duplicates found based on columns: ['encounter_block', 'recorded_date', 'recorded_hour'].


In [30]:
# 5) Add time_from_vent & 4-hr “cool-off”
hourly_seq_block['time_from_vent'] = hourly_seq_block.groupby('encounter_block').cumcount()
hourly_seq_block['time_from_vent_adjusted'] = np.where(
    hourly_seq_block['time_from_vent'] < 4, -1, hourly_seq_block['time_from_vent'] - 4
)

In [31]:
# 6) Combine with actual vent usage by hour
resp_stitched_imv = resp_stitched_imv[resp_stitched_imv['encounter_block'].isin(all_ids['encounter_block'])]
hourly_vent_block = resp_stitched_imv.groupby(['encounter_block','recorded_date','recorded_hour']).agg(
    min_fio2_set=('fio2_set','min'),
    max_fio2_set=('fio2_set','max'),
    min_peep_set=('peep_set','min'),
    max_peep_set=('peep_set','max'),
    min_lpm_set=('lpm_set', 'min'),
    max_lpm_set=('lpm_set', 'max'),
    min_resp_rate_obs=('resp_rate_obs', 'min'),
    max_resp_rate_obs=('resp_rate_obs', 'max'),
    hourly_trach=('tracheostomy', lambda x: 1 if x.max()==1 else 0), # 1 if the any value within that hour is 1
    hourly_on_vent=('on_vent','max'),
).reset_index()

In [32]:
# Find encounter_blocks that are in hourly_seq_block but not in hourly_vent_block and vice versa
seq_blocks = set(hourly_seq_block['encounter_block'].unique())
vent_blocks = set(hourly_vent_block['encounter_block'].unique())

blocks_in_seq_not_vent = seq_blocks - vent_blocks
blocks_in_vent_not_seq = vent_blocks - seq_blocks

print("Blocks in hourly_seq_block but not in hourly_vent_block:", len(blocks_in_seq_not_vent))
if len(blocks_in_seq_not_vent) > 0:
    print(sorted(list(blocks_in_seq_not_vent)))

print("\nBlocks in hourly_vent_block but not in hourly_seq_block:", len(blocks_in_vent_not_seq))
if len(blocks_in_vent_not_seq) > 0:
    print(sorted(list(blocks_in_vent_not_seq)))


Blocks in hourly_seq_block but not in hourly_vent_block: 0

Blocks in hourly_vent_block but not in hourly_seq_block: 1
[43820]


In [46]:
# encounter block 43820  hosp= 14309874 not in vitals stitched
c = resp_support_raw[resp_support_raw['hospitalization_id']== '14309874']
c 

,hospitalization_id,recorded_dttm,device_name,device_category,mode_name,mode_category,tracheostomy,fio2_set,lpm_set,resp_rate_set,peep_set,resp_rate_obs
547092,14309874,2021-03-13 22:02:00-06:00,Bag Valve Mask,IMV,None,None,0.0,NaN,NaN,NaN,NaN,NaN
547093,14309874,2021-03-13 22:10:00-06:00,Bag Valve Mask,IMV,None,None,0.0,NaN,NaN,NaN,NaN,NaN


In [47]:
hospitalization_cohort[hospitalization_cohort['hospitalization_id']== '14309874']

,patient_id,hospitalization_id,admission_dttm,discharge_dttm,age_at_admission,discharge_name,discharge_category,admission_type_name,admission_type_category,zipcode_five_digit,zipcode_nine_digit,census_block_group,latitude,longitude
117857,2178952,14309874,2021-03-14 04:02:00,2021-03-14 10:00:00,48.0,Expired,Expired,Non-Health Care Facility Point of Origin,Long Term Care Hospital (LTACH),60619,None,None,None,None


In [138]:
final_df_block = pd.merge(
    hourly_seq_block, 
    hourly_vent_block,
    on=['encounter_block','recorded_date','recorded_hour'],
    how='left'
)

In [80]:
# 7) Count how many vent hours per block in the first 72 hours after first intubation,
#  Exclude <4 hours on vent in first 72 hours at block level- They cannot meaningfully be studied for early mobilization if they’re barely intubated.. including them could bias results
first_72_hours = final_df_block[(final_df_block['time_from_vent'] >= 0) & (final_df_block['time_from_vent'] < 72)]
vent_hours_per_block = first_72_hours.groupby('encounter_block')['hourly_on_vent'].sum()

In [ ]:
blocks_under_4 = vent_hours_per_block[vent_hours_per_block < 4].index
blocks_under_4_df = final_df_block[final_df_block['encounter_block'].isin(blocks_under_4)]
final_df_block = final_df_block[~final_df_block['encounter_block'].isin(blocks_under_4)]

strobe_counts['G_blocks_with_vent_4_or_more'] = final_df_block['encounter_block'].nunique()
strobe_counts['G_blocks_with_vent_less_than_4'] = len(blocks_under_4)
print(f"Unique encounter blocks with valid IMV start/end: {strobe_counts['G_blocks_with_vent_4_or_more']}")
print(f"Excluded {len(blocks_under_4)} encounter blocks with <4 vent hours in first 72 hours of intubation.\n")

# 8) Exclude blocks with early trach in first 72
trach_flag_block = first_72_hours.groupby('encounter_block')['hourly_trach'].max()
blocks_with_trach = trach_flag_block[trach_flag_block==1].index

final_df_block = final_df_block[~final_df_block['encounter_block'].isin(blocks_with_trach)]
print(f"Excluded {len(blocks_with_trach)} encounter blocks with trach in first 72 hours of intubation.\n")

strobe_counts['G_final_blocks_without_trach'] = final_df_block['encounter_block'].nunique()
strobe_counts['G_final_blocks_with_trach'] = len(blocks_with_trach)
print(f"Final cohort size (unique blocks) after all exclusions: {strobe_counts['G_final_blocks_without_trach']}")

In [ ]:
strobe_counts

In [40]:
final_df = pd.merge(
    final_df_block,
    all_ids,
    on='encounter_block',
    how='left'
).reindex(columns=[
    'patient_id', 'hospitalization_id', 'encounter_block', 
    'recorded_dttm', 'recorded_date', 'recorded_hour',
    'time_from_vent', 'time_from_vent_adjusted',
    'min_fio2_set', 'max_fio2_set', 'min_peep_set', 'max_peep_set',
    'min_lpm_set', 'max_lpm_set', 'min_resp_rate_obs', 'max_resp_rate_obs',
    'hourly_trach', 'hourly_on_vent'
])

In [ ]:
# Check for duplicates
key_cols = ['encounter_block', 'recorded_date', 'recorded_hour']
duplicates = final_df.duplicated(subset=key_cols).sum()
print(f"Number of duplicate rows: {duplicates}")

In [ ]:
all_ids = all_ids[all_ids['encounter_block'].isin(final_df['encounter_block'])]
all_ids.shape

## Hourly Vitals

In [43]:
## get height , weight to calculate bmi
# Filter vitals to include only height and weight
vitals_bmi = vitals_stitched[vitals_stitched['vital_category'].isin(['weight_kg', 'height_cm'])].copy()

# Remove outliers
# Extract the min/max from the config
min_height, max_height = outlier_cfg['height_cm']
min_weight, max_weight = outlier_cfg['weight_kg']

# For height rows: set out-of-range to NaN
is_height = vitals_bmi['vital_category'] == 'height_cm'
height_mask_low  = is_height & (vitals_bmi['vital_value'] < min_height)
height_mask_high = is_height & (vitals_bmi['vital_value'] > max_height)
vitals_bmi.loc[height_mask_low | height_mask_high, 'vital_value'] = np.nan

# For weight rows: set out-of-range to NaN
is_weight = vitals_bmi['vital_category'] == 'weight_kg'
weight_mask_low  = is_weight & (vitals_bmi['vital_value'] < min_weight)
weight_mask_high = is_weight & (vitals_bmi['vital_value'] > max_weight)
vitals_bmi.loc[weight_mask_low | weight_mask_high, 'vital_value'] = np.nan

# Merge with vent_start_end to get ventilation start time
vitals_bmi = vitals_bmi.merge(
    block_vent_times[['encounter_block','vent_start_time']],
    on='encounter_block',
    how='left'
)

# Calculate time difference between recorded_dttm and vent_start_time
vitals_bmi['recorded_dttm'] = pd.to_datetime(vitals_bmi['recorded_dttm'])
vitals_bmi['vent_start_time'] = pd.to_datetime(vitals_bmi['vent_start_time'])
vitals_bmi['time_diff'] = (vitals_bmi['recorded_dttm'] - vitals_bmi['vent_start_time']).dt.total_seconds() / 3600  # in hours

# Define whether measurement is before or after vent_start_time
vitals_bmi['before_vent_start'] = (vitals_bmi['time_diff'] <= 0).astype(int)

# Calculate absolute time difference
vitals_bmi['abs_time_diff'] = vitals_bmi['time_diff'].abs()

# Sort data to prioritize measurements before vent start and closest in time
vitals_bmi = vitals_bmi.sort_values(['encounter_block', 'vital_category', 'before_vent_start', 'abs_time_diff'], 
                                    ascending=[True, True, False, True])

# Drop duplicates to keep the closest measurement for each vital_category per encounter block
vitals_bmi = vitals_bmi.drop_duplicates(subset=['encounter_block', 'vital_category'], keep='first')

# Pivot to get height and weight per encounter block
vitals_bmi_pivot = vitals_bmi.pivot(index='encounter_block', 
                                    columns='vital_category', 
                                    values='vital_value'
                                    ).reset_index()

# Calculate BMI
vitals_bmi_pivot['bmi'] = vitals_bmi_pivot['weight_kg'] / ((vitals_bmi_pivot['height_cm'] / 100) ** 2)

In [ ]:
# Ensure 'recorded_dttm' is datetime
vitals_stitched['recorded_dttm'] = pd.to_datetime(vitals_stitched['recorded_dttm'])

# Extract 'recorded_date' and 'recorded_hour' from recorded_dttm
vitals_stitched['recorded_date'] = vitals_stitched['recorded_dttm'].dt.date
vitals_stitched['recorded_hour'] = vitals_stitched['recorded_dttm'].dt.hour

# Check if 'map' exists
if 'map' not in vitals_stitched['vital_category'].unique():
    print("map is not present, so we'll calculate it...")
    # 1) Filter for sbp & dbp
    sbp_dbp = vitals_stitched[vitals_stitched['vital_category'].isin(['sbp','dbp'])].copy()
    
    # 2) Pivot at the encounter_block + recorded_dttm level
    sbp_dbp_pivot = sbp_dbp.pivot_table(
        index=['encounter_block','recorded_dttm'],
        columns='vital_category',
        values='vital_value'
    ).reset_index()
    
    # 3) Drop any row missing sbp or dbp
    sbp_dbp_pivot = sbp_dbp_pivot.dropna(subset=['sbp','dbp'])
    
    # 4) Calculate MAP
    sbp_dbp_pivot['map'] = (sbp_dbp_pivot['sbp'] + 2*sbp_dbp_pivot['dbp']) / 3
    
    # 5) Build a DataFrame for map
    map_vitals = sbp_dbp_pivot[['encounter_block','recorded_dttm','map']].copy()
    map_vitals['vital_category'] = 'map'
    map_vitals['vital_value'] = map_vitals['map']
    
    # Also add recorded_date/hour
    map_vitals['recorded_date'] = map_vitals['recorded_dttm'].dt.date
    map_vitals['recorded_hour'] = map_vitals['recorded_dttm'].dt.hour
    
    # Keep only the needed columns
    map_vitals = map_vitals[[
        'encounter_block','recorded_dttm','recorded_date','recorded_hour','vital_category','vital_value'
    ]]
    
    # 6) Append 'map' to the main vitals_stitched DataFrame
    vitals_stitched = pd.concat([vitals_stitched, map_vitals], ignore_index=True)
    print("...map was calculated and appended to vitals_stitched.")


#Compute min/max vitals  at the BLOCK level
# group by encounter_block + recorded_date + recorded_hour + vital_category
vitals_min_max = vitals_stitched.groupby(
    ['encounter_block','recorded_date','recorded_hour','vital_category']
).agg(
    min_val=('vital_value','min'),
    max_val=('vital_value','max')
).reset_index()

# 3) Pivot so each row is unique by (encounter_block, recorded_date, recorded_hour),
#    with columns like min_sbp, max_sbp, min_map, max_map, etc.
vitals_pivot = vitals_min_max.pivot_table(
    index=['encounter_block','recorded_date','recorded_hour'],
    columns='vital_category',
    values=['min_val','max_val']
).reset_index()

# Flatten the multi-level columns
vitals_pivot.columns = [
    '_'.join(col).rstrip('_') if isinstance(col, tuple) else col 
    for col in vitals_pivot.columns
]

#  Rename columns for clarity
rename_dict = {}
for c in vitals_pivot.columns:
    if c.startswith('min_val_'):
        rename_dict[c] = c.replace('min_val_','min_')
    elif c.startswith('max_val_'):
        rename_dict[c] = c.replace('max_val_','max_')

vitals_pivot = vitals_pivot.rename(columns=rename_dict)

# The resulting columns might look like:
# ['encounter_block','recorded_date','recorded_hour',
#  'min_sbp','max_sbp','min_map','max_map','min_resp_rate','max_resp_rate', etc.]

print("Finished creating block-level min/max vitals pivot:")
vitals_pivot.head(10)

In [45]:
# merge vitals with final_df
final_df = pd.merge(final_df, vitals_pivot, on=['encounter_block', 'recorded_date', 'recorded_hour'], 
                   how='left')

In [ ]:
## confirm duplicates don't exist
checkpoint_vitals = pyCLIF.remove_duplicates(final_df, [
    'encounter_block','recorded_date', 'recorded_hour'
], 'final_df')
del checkpoint_vitals

## Hourly Meds

* Handle med dose unit conversion for all vasoactives
* Calculate NE equivalent levels using "norepinephrine", "epinephrine", "phenylephrine", "vasopressin", "dopamine",  "angiotensin"
* Create flags for "nicardipine", "nitroprusside", "clevidipine" for the red criteria under consensus criteria
* Identify encounters on paralytics - cisatracurium, vecuronium, rocuronium- and create flags for each of these paralytic meds. These patients will not be considered eligible for mobilization during the hour they were receiving paralytic medication. 


In [ ]:
# Import clif continuous meds for the cohort on vent during the required time period
meds_filters = {
    'hospitalization_id': all_ids['hospitalization_id'].unique().tolist(),
    'med_category': meds_of_interest
}
meds = pyCLIF.load_data('clif_medication_admin_continuous', columns=meds_required_columns, filters=meds_filters)
print("unique encounters in meds", pyCLIF.count_unique_encounters(meds))

In [93]:
# ensure correct format
meds['hospitalization_id']= meds['hospitalization_id'].astype(str)
meds['med_dose_unit'] = meds['med_dose_unit'].str.lower()
meds['admin_dttm'] = pd.to_datetime(meds['admin_dttm'], format='%Y-%m-%d %H:%M:%S')
meds['med_dose'] = pd.to_numeric(meds['med_dose'], errors='coerce')
# Create 'date' and 'hour_of_day' columns
meds['recorded_date'] = meds['admin_dttm'].dt.date
meds['recorded_hour'] = meds['admin_dttm'].dt.hour

In [ ]:
# Create a summary table for each med_category
summary_table = meds.groupby('med_category').agg(
    total_N=('med_category', 'size'),
    min=('med_dose', 'min'),
    max=('med_dose', 'max'),
    first_quantile=('med_dose', lambda x: x.quantile(0.25)),
    second_quantile=('med_dose', lambda x: x.quantile(0.5)),
    third_quantile=('med_dose', lambda x: x.quantile(0.75)),
    missing_values=('med_dose', lambda x: x.isna().sum())
).reset_index()

## check the distrbituon of required continuous meds
summary_table

In [ ]:
# Create a summary table for each med_category and med_dose_unit combination
summary_table = meds.groupby(['med_category', 'med_dose_unit']).agg(
    total_N=('med_category', 'size'),
    min=('med_dose', 'min'),
    max=('med_dose', 'max'),
    first_quantile=('med_dose', lambda x: x.quantile(0.25)),
    second_quantile=('med_dose', lambda x: x.quantile(0.5)),
    third_quantile=('med_dose', lambda x: x.quantile(0.75)),
    missing_values=('med_dose', lambda x: x.isna().sum())
).reset_index()

## check the distrbituon of required continuous meds
summary_table

In [ ]:
# Group by med_category and med_dose_unit
grouped_data = meds.groupby(['med_category', 'med_dose_unit'])

# Dynamically determine the number of required subplots
n_plots = len(grouped_data.groups.keys())
n_cols = 4
n_rows = (n_plots + n_cols - 1) // n_cols  # Round up to determine rows

fig, axs = plt.subplots(n_rows, n_cols, figsize=(20, n_rows * 5))

# Flatten the axs array for easier indexing
axs = axs.flatten()

# Loop through each group and plot the histogram
for i, ((med_category, med_dose_unit), group) in enumerate(grouped_data):
    ax = axs[i]
    ax.hist(group['med_dose'], bins=20, alpha=0.7, label=f"N = {len(group)}")
    ax.set_title(f"{med_category} - {med_dose_unit}")
    ax.set_xlabel('Med Dose')
    ax.set_ylabel('Frequency')
    ax.legend()
    ax.grid(True)

# Hide any unused axes
for j in range(i + 1, len(axs)):
    axs[j].axis('off')

plt.tight_layout()
plt.show()

In [100]:
# Define medications and their unit conversion information
meds_list = [
    "norepinephrine", "epinephrine", "phenylephrine",
    "vasopressin", "dopamine", "angiotensin", "metaraminol"
]

med_unit_info = {
    'norepinephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'epinephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'phenylephrine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'dopamine': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mcg/kg/min', 'mcg/kg/hr', 'mg/kg/hr', 'mcg/min', 'mg/hr'],
    },
    'metaraminol': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['mg/hr', 'mcg/min'],
    },
    'angiotensin': {
        'required_unit': 'mcg/kg/min',
        'acceptable_units': ['ng/kg/min', 'ng/kg/hr'],
    },
    'vasopressin': {
        'required_unit': 'units/min',
        'acceptable_units': ['units/min', 'units/hr', 'milliunits/min', 'milliunits/hr'],
    },
}

In [ ]:
# Check the med_dose_unit for each med_category in the meds table
med_dose_unit_check = meds.groupby(['med_category', 'med_dose_unit']).size().reset_index(name='count')

# Create a new column to flag invalid dose units
def check_dose_unit(row):
    med_category = row['med_category']
    med_dose_unit = row['med_dose_unit']
    # Check if med_category exists in med_unit_info
    if med_category in med_unit_info:
        # Check if med_dose_unit is in the acceptable units
        if med_dose_unit in med_unit_info[med_category]['acceptable_units']:
            return "Valid"
        else:
            return "Not an acceptable unit"
    else:
        return "Not a vasoactive"

# Apply the function to the DataFrame
med_dose_unit_check['unit_validity'] = med_dose_unit_check.apply(check_dose_unit, axis=1)

# # Optional: Filter for invalid units
invalid_units = med_dose_unit_check[med_dose_unit_check['unit_validity'] == 'Not an acceptable unit']
print("Invalid units. These will be dropped:\n")
print(invalid_units)

In [106]:
# ## Norepinephrine equivalent calculation
# Goradia S, Sardaneh AA, Narayan SW, Penm J, Patanwala AE. Vasopressor dose equivalence: 
# A scoping review and suggested formula. J Crit Care. 2021 Feb;61:233-240. doi: 10.1016/j.jcrc.2020.11.002. Epub 2020 Nov 14. PMID: 33220576.

meds_list = [
    "norepinephrine", "epinephrine", "phenylephrine", 
    "vasopressin", "dopamine",  
    "angiotensin"
]

# Function to check if 'med_dose_unit' contains '/hr' or '/min'
def has_per_hour_or_min(unit):
    if pd.isnull(unit):
        return False
    unit = unit.lower()
    return '/hr' in unit or '/min' in unit

# Filter meds to include only rows with '/hr' or '/min' in 'med_dose_unit'
meds_filtered = meds[meds['med_dose_unit'].apply(has_per_hour_or_min)].copy()

ne_df = meds_filtered[meds_filtered['med_category'].isin(meds_list)].copy()

In [ ]:
# **2. Convert Medication Doses to Required Units**
### Function to get conversion factor for each medication
def get_conversion_factor(med_category, med_dose_unit, weight_kg):
    med_info = med_unit_info.get(med_category, None)
    if not med_info:
        # Medication not in the list
        return None
    required_unit = med_info['required_unit']
    acceptable_units = med_info['acceptable_units']
    med_dose_unit = med_dose_unit.lower()
    if med_category in ['norepinephrine', 'epinephrine', 'phenylephrine', 'dopamine', 'metaraminol']:
        # Required unit: mcg/kg/min
        if med_dose_unit == 'mcg/kg/min':
            factor = 1.0
        elif med_dose_unit == 'mcg/kg/hr':
            factor = 1 / 60
        elif med_dose_unit == 'mg/kg/hr':
            factor = 1000 / 60
        elif med_dose_unit == 'mcg/min':
            factor = 1 / weight_kg
        elif med_dose_unit == 'mg/hr':
            factor = 1000 / 60 / weight_kg
        else:
            return None
    elif med_category == 'angiotensin':
        # Required unit: mcg/kg/min
        if med_dose_unit == 'ng/kg/min':
            factor = 1 / 1000
        elif med_dose_unit == 'ng/kg/hr':
            factor = 1 / 1000 / 60
        else:
            return None
    elif med_category == 'vasopressin':
        # Required unit: units/min
        if med_dose_unit == 'units/min':
            factor = 1.0
        elif med_dose_unit == 'units/hr':
            factor = 1 / 60
        elif med_dose_unit == 'milliunits/min':
            factor = 1 / 1000
        elif med_dose_unit == 'milliunits/hr':
            factor = 1 / 1000 / 60
        else:
            return None
    else:
        return None
    return factor

# Merge weight_kg into meds_filtered (assuming 'vitals_bmi_pivot' is available)
meds_filtered = meds_filtered.merge(vitals_bmi_pivot[['hospitalization_id', 'weight_kg']], on='hospitalization_id', how='left')

# Remove rows with missing weight_kg
meds_filtered = meds_filtered[~meds_filtered['weight_kg'].isnull()].copy()

# Function to convert doses
def convert_dose(row):
    med_category = row['med_category']
    med_dose = row['med_dose']
    med_dose_unit = row['med_dose_unit']
    weight_kg = row['weight_kg']
    factor = get_conversion_factor(med_category, med_dose_unit, weight_kg)
    if factor is None:
        return np.nan
    return med_dose * factor

# Apply the conversion to get 'med_dose_converted'
meds_filtered['med_dose_converted'] = meds_filtered.apply(convert_dose, axis=1)

# Drop rows with NaN in 'med_dose_converted' (unrecognized units)
meds_filtered = meds_filtered[~meds_filtered['med_dose_converted'].isnull()].copy()

# Define acceptable dose ranges
med_dose_ranges = {
    'norepinephrine': (0.01, 3),
    'epinephrine': (0.01, 0.1),
    'phenylephrine': (0.1, 5),
    'dopamine': (2, 20),
    'metaraminol': (0.5, 10),  
    'angiotensin': (0.02, 0.2), 
    'vasopressin': (0.01, 0.1),  
}

# Function to check if dose is within range
def is_dose_within_range(row, outlier_dict):
    '''
    Check if med_dose_converted is within the outlier-configured range for this med_category.
    Parameters:
        row (pd.Series): A row from a DataFrame, must include 'med_category' and 'med_dose_converted'.
        outlier_dict (dict): Dictionary of min/max pairs from outlier_config.json.
    Returns:
        bool: True if the dose is within range or if med_category is not found, False otherwise.
    '''
    med_category = row['med_category']
    med_dose_converted = row['med_dose_converted']
    dose_range = outlier_dict.get(med_category, None)
    if dose_range is None:
        return False
    min_dose, max_dose = dose_range
    return min_dose <= med_dose_converted <= max_dose

# Filter doses within acceptable ranges
meds_filtered = meds_filtered[meds_filtered.apply(is_dose_within_range, axis=1, args=(outlier_cfg,))].copy()

# **4. Flag Medications Not in the Dataset**

for med in meds_list:
    if med not in meds_filtered['med_category'].unique():
        print(f"{med} is not in the dataset.")

# Pivot and Aggregate the Data**

# Create 'recorded_date' and 'recorded_hour' columns
meds_filtered['admin_dttm'] = pd.to_datetime(meds_filtered['admin_dttm'])
meds_filtered['recorded_date'] = meds_filtered['admin_dttm'].dt.date
meds_filtered['recorded_hour'] = meds_filtered['admin_dttm'].dt.hour

# Group and aggregate doses
group_cols = ['hospitalization_id', 'recorded_date', 'recorded_hour', 'med_category']
dose_agg = meds_filtered.groupby(group_cols)['med_dose_converted'].agg(['min', 'max']).reset_index()

# Pivot to have medications as columns
dose_pivot_min = dose_agg.pivot_table(index=['hospitalization_id', 'recorded_date', 'recorded_hour'], columns='med_category', values='min').reset_index()
dose_pivot_max = dose_agg.pivot_table(index=['hospitalization_id', 'recorded_date', 'recorded_hour'], columns='med_category', values='max').reset_index()

# Rename columns to indicate min and max
dose_pivot_min.columns = ['hospitalization_id', 'recorded_date', 'recorded_hour'] + ['min_' + col for col in dose_pivot_min.columns if col not in ['hospitalization_id', 'recorded_date', 'recorded_hour']]
dose_pivot_max.columns = ['hospitalization_id', 'recorded_date', 'recorded_hour'] + ['max_' + col for col in dose_pivot_max.columns if col not in ['hospitalization_id', 'recorded_date', 'recorded_hour']]

# Merge min and max DataFrames
dose_pivot = pd.merge(dose_pivot_min, dose_pivot_max, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], how='outer')

# **6. Calculate Norepinephrine Equivalents**

# Replace NaN with 0 for calculations
dose_pivot.fillna(0, inplace=True)

# Calculate NE min
dose_pivot['ne_calc_min'] = (
    dose_pivot.get('min_norepinephrine', 0) +
    dose_pivot.get('min_epinephrine', 0) +
    dose_pivot.get('min_phenylephrine', 0) / 10 +
    dose_pivot.get('min_dopamine', 0) / 100 +
    dose_pivot.get('min_metaraminol', 0) / 8 +
    dose_pivot.get('min_vasopressin', 0) * 2.5 +
    dose_pivot.get('min_angiotensin', 0) * 10
)

# Calculate NE max
dose_pivot['ne_calc_max'] = (
    dose_pivot.get('max_norepinephrine', 0) +
    dose_pivot.get('max_epinephrine', 0) +
    dose_pivot.get('max_phenylephrine', 0) / 10 +
    dose_pivot.get('max_dopamine', 0) / 100 +
    dose_pivot.get('max_metaraminol', 0) / 8 +
    dose_pivot.get('max_vasopressin', 0) * 2.5 +
    dose_pivot.get('max_angiotensin', 0) * 10
)

# **7. Prepare the Final Dataset**
# Keep only the required columns
ne_calc_df = dose_pivot[['hospitalization_id', 'recorded_date', 
                         'recorded_hour', 
                         'ne_calc_min', 'ne_calc_max']].drop_duplicates(subset=['hospitalization_id', 'recorded_date', 'recorded_hour'])

In [65]:
final_df = pd.merge(final_df, ne_calc_df, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], how='left')

In [68]:
red_meds_list = [
    "nicardipine", "nitroprusside", "clevidipine"
]

# Filter meds_filtered for the medications in red_meds_list
red_meds_df = meds[meds['med_category'].isin(red_meds_list)].copy()

# Create a flag for each medication in red_meds_list
for med in red_meds_list:
    # Create a flag that is 1 if the medication was administered in that hour, 0 otherwise
    red_meds_df[med + '_flag'] = np.where(red_meds_df['med_category'] == med, 1, 0).astype(int)

# Aggregate to get the maximum value for each flag (per hospitalization_id, recorded_date, recorded_hour)
# This ensures that if the medication was administered even once in the hour, the flag is 1
red_meds_flags = red_meds_df.groupby(['hospitalization_id', 'recorded_date', 'recorded_hour']).agg(
    {med + '_flag': 'max' for med in red_meds_list}
).reset_index()

#  combine all flags into a single 'red_meds_flag', you can do so like this:
red_meds_flags['red_meds_flag'] = red_meds_flags[[med + '_flag' for med in red_meds_list]].max(axis=1)

# Select the relevant columns
red_meds_flags_final = red_meds_flags[[
    'hospitalization_id', 'recorded_date', 'recorded_hour',
    'nicardipine_flag', 'nitroprusside_flag',
    'clevidipine_flag', 'red_meds_flag'
]].drop_duplicates(subset=['hospitalization_id', 'recorded_date', 'recorded_hour'])

red_meds_flags_final['nicardipine_flag'] = red_meds_flags_final['nicardipine_flag'].astype(int)
red_meds_flags_final['nitroprusside_flag'] = red_meds_flags_final['nitroprusside_flag'].astype(int)
red_meds_flags_final['clevidipine_flag'] = red_meds_flags_final['clevidipine_flag'].astype(int)
red_meds_flags_final['red_meds_flag'] = red_meds_flags_final['red_meds_flag'].astype(int)

In [69]:
final_df = pd.merge(final_df, red_meds_flags_final, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], how='left')

In [66]:
paralytics_list = [
    "cisatracurium", "vecuronium", "rocuronium" 
]

# Filter meds_filtered for the medications in paralytics_list
paralytics_df = meds[meds['med_category'].isin(paralytics_list)].copy()

# Create a flag for each medication in paralytics_list
for med in paralytics_list:
    # Create a flag that is 1 if the medication was administered in that hour, 0 otherwise
    paralytics_df[med + '_flag'] = np.where(paralytics_df['med_category'] == med, 1, 0).astype(int)

# Aggregate to get the maximum value for each flag (per hospitalization_id, recorded_date, recorded_hour)
# This ensures that if the medication was administered even once in the hour, the flag is 1
paralytics_flags = paralytics_df.groupby(['hospitalization_id', 'recorded_date', 'recorded_hour']).agg(
    {med + '_flag': 'max' for med in paralytics_list}
).reset_index()

#  combine all flags into a single 'paralytics_flag', you can do so like this:
paralytics_flags['paralytics_flag'] = paralytics_flags[[med + '_flag' for med in paralytics_list]].max(axis=1)

# Select the relevant columns
paralytics_flags_final = paralytics_flags[[
    'hospitalization_id', 'recorded_date', 'recorded_hour',
    'cisatracurium_flag', 'vecuronium_flag',
    'rocuronium_flag', 'paralytics_flag'
]].drop_duplicates(subset=['hospitalization_id', 'recorded_date', 'recorded_hour'])

paralytics_flags_final['cisatracurium_flag'] = paralytics_flags_final['cisatracurium_flag'].astype(int)
paralytics_flags_final['vecuronium_flag'] = paralytics_flags_final['vecuronium_flag'].astype(int)
paralytics_flags_final['rocuronium_flag'] = paralytics_flags_final['rocuronium_flag'].astype(int)
paralytics_flags_final['paralytics_flag'] = paralytics_flags_final['paralytics_flag'].astype(int)

In [67]:
final_df = pd.merge(final_df, paralytics_flags_final, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], how='left')

## Hourly Lab

Get most recent lactate defined as closest lab result time to the start of first intubation event

In [ ]:
# Import clif continuous meds and clif labs table for the cohort on vent during the required time period
labs_filters = {
    'hospitalization_id': cohort_ids,
    'lab_category': labs_of_interest
}
labs = pyCLIF.load_data('clif_labs', columns=labs_required_columns, filters=labs_filters)
print("unique encounters in labs", pyCLIF.count_unique_encounters(labs))
labs['hospitalization_id']= labs['hospitalization_id'].astype(str)

In [53]:
labs['lab_result_dttm'] = pd.to_datetime(labs['lab_result_dttm'])
labs['recorded_hour'] = labs['lab_result_dttm'].dt.hour
labs['recorded_date'] = labs['lab_result_dttm'].dt.date

lactate_df = pd.merge(labs, vent_start_end, on='hospitalization_id', how='left')
lactate_df['time_since_vent_start_hours'] = (
    (lactate_df['lab_result_dttm'] - lactate_df['vent_start_time']).dt.total_seconds() / 3600
)

# Calculate the absolute time difference between lab_result_dttm and vent_start_time in hours
lactate_df['time_diff_hours'] = abs((lactate_df['lab_result_dttm'] - lactate_df['vent_start_time']).dt.total_seconds() / 3600)

# Filter for observations within the first 72 hours since vent_start_time
lactate_df = lactate_df[(lactate_df['time_since_vent_start_hours'] >= 0) & 
                        (lactate_df['time_since_vent_start_hours'] <= 72)]

# Sort by hospitalization_id, recorded_hour, and time_diff_hours to find the closest measurement to vent_start_time
lactate_df = lactate_df.sort_values(by=['hospitalization_id', 'recorded_date', 'recorded_hour', 'time_diff_hours'])

# Group by hospitalization_id and recorded_hour, and get the first row in each group (which is the closest measurement)
# closest lactate measurement is defined as closest to the vent_start_time in that hour. 
closest_lactate_df = lactate_df.groupby(['hospitalization_id', 'recorded_date','recorded_hour']).first().reset_index()

labs_final = closest_lactate_df[['hospitalization_id', 'recorded_date', 'recorded_hour', 'lab_value_numeric']].copy()

# Rename the 'lab_value_numeric' column to 'lactate'
labs_final = labs_final.rename(columns={'lab_value_numeric': 'lactate'})

final_df = pd.merge(final_df, labs_final, on=['hospitalization_id', 'recorded_date', 'recorded_hour'], 
                   how='left')

In [ ]:
checkpoint_labs= pyCLIF.remove_duplicates(final_df, [
    'hospitalization_id', 'recorded_date', 'recorded_hour'
], 'final_df')
del checkpoint_labs

In [ ]:
final_df.columns

## Write analysis dataset 

In [56]:
final_df.to_parquet('../output/intermediate/final_df.parquet')
vent_start_end.to_parquet('../output/intermediate/vent_start_end.parquet')
final_df['hospitalization_id'].to_csv('../output/intermediate/cohort_ids.csv', index=False)